In [28]:
import pandas as pd
import numpy as np

In [29]:
# Load the dataset
df = pd.read_csv('./Data/train.csv')

In [30]:
def detect_column_type(df, threshold=15):
    column_types = {}
    for col in df.columns:
        unique_vals = df[col].nunique()
        dtype = df[col].dtype

        if pd.api.types.is_numeric_dtype(dtype):
            # Treat as continuous if unique values are more than threshold
            column_types[col] = 'Continuous' if unique_vals > threshold else 'Categorical'
        else:
            column_types[col] = 'Categorical'
    
    return pd.DataFrame.from_dict(column_types, orient='index', columns=['Feature_Type'])

# Usage
column_type_df = detect_column_type(df)
print(column_type_df)


                  Feature_Type
Unnamed: 0          Continuous
Name               Categorical
Location           Categorical
Year                Continuous
Kilometers_Driven   Continuous
Fuel_Type          Categorical
Transmission       Categorical
Owner_Type         Categorical
Mileage            Categorical
Engine             Categorical
Power              Categorical
Seats              Categorical
New_Price          Categorical
Price               Continuous


In [31]:
print(df.head(5))

   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Fuel_Type Transmission Owner_Type     Mileage   Engine  \
0              41000    Diesel       Manual      First  19.67 kmpl  1582 CC   
1              46000    Petrol       Manual      First    13 km/kg  1199 CC   
2              87000    Diesel       Manual      First  20.77 kmpl  1248 CC   
3              40670    Diesel    Automatic     Second   15.2 kmpl  1968 CC   
4              86999    Diesel       Manual      First  23.08 kmpl  1461 CC   

       Power  Seats  New_Price  Price  
0  126.2 bhp    5.0        NaN  12.50  
1   88.7 bhp    5.0  8.61 Lakh

#### a. Look for the missing values in all the columns and either impute them (replace with mean,median, or mode) or drop them. Justify your action for this task

In [32]:
# Check missing values
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values)

df.shape


Missing values per column:
 Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


(5847, 14)

#### Total datapoints are 5847, and the null values for new_price is more that 5000, hence removing is suggestable

In [33]:

df = df.drop(columns=['Unnamed: 0', 'New_Price'])  # Drop unnecessary or mostly missing columns

# Double check for missing values
print("Missing values after handling:\n", df.isnull().sum())


Missing values after handling:
 Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               2
Engine               36
Power                36
Seats                38
Price                 0
dtype: int64


#### b. Remove the units from some of the attributes and only keep the numerical values

In [34]:
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,4.50
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,3.50


In [35]:
# Clean Mileage
df['Mileage'] = df['Mileage'].str.replace(' kmpl', '', regex=False)
df['Mileage'] = df['Mileage'].str.replace(' km/kg', '', regex=False)

# Clean Engine
df['Engine'] = df['Engine'].str.replace(' CC', '', regex=False)

# Clean Power
df['Power'] = df['Power'].str.replace(' bhp', '', regex=False)

# Convert all to numeric
df[['Mileage', 'Engine', 'Power']] = df[['Mileage', 'Engine', 'Power']].apply(pd.to_numeric, errors='coerce')

df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,12.50
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13.00,1199.0,88.70,5.0,4.50
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,17.74
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461.0,63.10,5.0,3.50


#### Additionally I can add the following code to handle the remaininng null values for other colmuns

In [36]:
# Impute Mileage, Engine, Power, Seats with median
for col in ['Mileage', 'Engine', 'Power', 'Seats']:
    # First, remove units to convert them to numeric (we'll do this in next step too, but needed here to impute correctly)
    df[col] = pd.to_numeric(df[col], errors='coerce')
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)

print("Missing values after handling:\n", df.isnull().sum())

Missing values after handling:
 Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64


##### C. Change the categorical variables 

In [37]:
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)
df.head()

,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,12.50,False,False,True
1,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,4.50,False,True,True
2,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,False,False,True
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,17.74,False,False,False
4,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,3.50,False,False,True


##### d. Create one more feature and add this column to the dataset 

In [38]:
df['Car_Age'] = 2025 - df['Year']
df.head()

,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,12.50,False,False,True,10
1,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,4.50,False,True,True,14
2,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,False,False,True,13
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,17.74,False,False,False,12
4,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,3.50,False,False,True,12


##### e. Perform select, filter, rename, mutate, arrange and summarize with group by operations (or their equivalent operations in python) on this dataset.

In [39]:
print(df.columns)

Index(['Name', 'Location', 'Year', 'Kilometers_Driven', 'Owner_Type',
       'Mileage', 'Engine', 'Power', 'Seats', 'Price', 'Fuel_Type_Electric',
       'Fuel_Type_Petrol', 'Transmission_Manual', 'Car_Age'],
      dtype='object')


In [40]:
selected_df = df[['Name', 'Location', 'Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Price']]
print("Selected Columns:\n", selected_df.head(), "\n")

# --- Filter: cars from year >= 2015 and Price > 10 ---
filtered_df = df[(df['Year'] >= 2015) & (df['Price'] > 10)]
print("Filtered Cars (Year >= 2015 and Price > 10 Lakh):\n", filtered_df[['Name', 'Year', 'Price']].head(), "\n")

# --- Rename: rename 'Kilometers_Driven' to 'KMs' ---
df.rename(columns={'Kilometers_Driven': 'KMs'}, inplace=True)
print("Columns after renaming:\n", df.columns, "\n")

# --- Mutate: create power-to-engine ratio ---
df['Power_to_Engine'] = df['Power'] / df['Engine']
print("Power to Engine Ratio (first 5 rows):\n", df[['Power', 'Engine', 'Power_to_Engine']].head(), "\n")

# --- Arrange: sort by Price descending ---
arranged_df = df.sort_values(by='Price', ascending=False)
print("Top 5 Most Expensive Cars:\n", arranged_df[['Name', 'Price']].head(), "\n")

# --- Summarize: average price by Location ---
summary_df = df.groupby('Location')['Price'].agg(['mean', 'count']).reset_index()
summary_df.rename(columns={'mean': 'Avg_Price', 'count': 'Total_Cars'}, inplace=True)
print("Average Price by Location:\n", summary_df.head(), "\n")

Selected Columns:
                                Name    Location  Year  Kilometers_Driven  \
0  Hyundai Creta 1.6 CRDi SX Option        Pune  2015              41000   
1                      Honda Jazz V     Chennai  2011              46000   
2                 Maruti Ertiga VDI     Chennai  2012              87000   
3   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013              40670   
4            Nissan Micra Diesel XV      Jaipur  2013              86999   

   Mileage  Engine   Power  Price  
0    19.67  1582.0  126.20  12.50  
1    13.00  1199.0   88.70   4.50  
2    20.77  1248.0   88.76   6.00  
3    15.20  1968.0  140.80  17.74  
4    23.08  1461.0   63.10   3.50   

Filtered Cars (Year >= 2015 and Price > 10 Lakh):
                                     Name  Year  Price
0       Hyundai Creta 1.6 CRDi SX Option  2015  12.50
5      Toyota Innova Crysta 2.8 GX AT 8S  2016  17.50
20  Audi A6 2011-2015 35 TFSI Technology  2015  23.50
43             Toyota Corolla Altis G A